In [1]:
!pip install -q transformers torch bitsandbytes requests sentencepiece openai accelerate

In [2]:
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
from IPython.display import display, Markdown
from huggingface_hub import login
from openai import OpenAI

In [3]:
login(userdata.get('HF_TOKEN'))
openai_api_key = userdata.get('OPENAI_API_KEY')

In [4]:
openai = OpenAI(api_key=openai_api_key)


In [5]:
MODEL = 'whisper-1'
audio_filename = open('/content/denver_extract.mp3', 'rb')
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
def transcribe(audio_file):
  response = openai.audio.transcriptions.create(
      model = MODEL,
      file = audio_file,

  )
  return response.text


In [14]:
system_prompt = """You are an expert meeting‐minutes assistant. When given a raw audio transcription, your task is to
produce concise, well‐structured meeting minutes that include:
1. Meeting details (date, time, attendees, if mentioned)
2. Agenda items discussed
3. Key decisions made
4. Action items (with owners and due dates, if mentioned)
5. Any open questions or follow-up items
Preserve all factual content; do not add information not present in the transcript. Respond in Markdown.
"""


user_prompt = "Please generate meeting minutes from the following transcript and respond in markdown:\n " + transcribe(audio_filename)

In [15]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [16]:
LLAMA = "meta-llama/Llama-3.1-8B-Instruct"

In [17]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [18]:
def generate_response():
    tokenizer = AutoTokenizer.from_pretrained(LLAMA)
    tokenizer.pad_token = tokenizer.eos_token
    prompt = tokenizer.apply_chat_template(messages, tokenize=False)

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    model = AutoModelForCausalLM.from_pretrained(
        LLAMA,
        device_map="auto",
        quantization_config=quant_config
    )
    # Generate output (non-streamed, since we want to post-process it)
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=2000,
        pad_token_id=tokenizer.eos_token_id,
    )
    # Slice off the input to get only the newly generated part
    new_tokens = outputs[0][inputs["input_ids"].shape[-1]:]
    response = tokenizer.decode(new_tokens, skip_special_tokens=True)
    clean_response = response.strip()
    display(Markdown(clean_response))

    del inputs, outputs, model, tokenizer
    torch.cuda.empty_cache()


In [19]:
generate_response()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

assistant

**Meeting Details**
* Date: October 9th
* Time: Not specified
* Attendees: Councilman Lopez, Councilman Clark, Councilwoman Ortega, Councilwoman Kinneach, Clerk, and other council members (11 present)

**Agenda Items Discussed**

* Approval of the minutes of the previous meeting (October 2nd)
* Council announcements
* Presentations
* Communications
* Proclamation 1127, an observance of the annual Indigenous Peoples Day in the City and County of Denver

**Key Decisions Made**

* Proclamation 1127, Series of 2017, an observance of the second annual Indigenous Peoples Day in the City and County of Denver, was adopted by the council.

**Action Items**

* None mentioned

**Open Questions or Follow-up Items**

* None mentioned

**Additional Notes**

* The council recognized the contributions of Indigenous people to the city's history, present, and future, and promoted the education of the Denver community about these historical and contemporary contributions.
* The proclamation was celebrated as a day of inclusivity and respect for all cultures, especially in a time of divisiveness.
* Councilman Lopez mentioned the importance of being proud of one's own culture without disrespecting others, quoting Cesar Chavez.
* Councilwoman Kinneach highlighted the importance of cultural preservation and environmental protection, mentioning the Native American community's efforts to defend public lands.